In [ ]:
%load_ext Cython
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import pandas as pd
import pykonal
import seispy

DTYPE_REAL = np.float64
EARTH_RADIUS = 6371.

def geo2sph(arr) -> np.ndarray:
    """
    Map Geographical coordinates to spherical coordinates.
    """
    geo = np.array(arr, dtype=DTYPE_REAL)
    sph = np.empty_like(geo)
    sph[..., 0] = EARTH_RADIUS - geo[..., 2]
    sph[..., 1] = np.pi / 2 - np.radians(geo[..., 0])
    sph[..., 2] = np.radians(geo[..., 1])
    return (sph)


def sph2geo(arr) -> np.ndarray:
    """
    Map spherical coordinates to geographic coordinates.
    """
    sph = np.array(arr, dtype=DTYPE_REAL)
    geo = np.empty_like(sph)
    geo[..., 0] = np.degrees(np.pi / 2 - sph[..., 1])
    geo[..., 1] = np.degrees(sph[..., 2])
    geo[..., 2] = EARTH_RADIUS - sph[..., 0]
    return (geo)

In [ ]:
# Load velocity model
grid = pykonal.fields.ScalarField3D(coord_sys="spherical")
with np.load("/home/malcolmw/google_drive/malcolm.white@usc.edu/data/velocity/White_et_al_2019a/White_et_al_2019a.regular.npz") as npz:
        grid.min_coords = npz["grid_parameters"][:3]
        grid.node_intervals = npz["grid_parameters"][3:6]
        grid.npts = npz["grid_parameters"][6:] + [1, 0, 0]
        pwave_velocity = np.append(npz["vp"], npz["vp"][[-1]], axis=0)
        swave_velocity = np.append(npz["vs"], npz["vs"][[-1]], axis=0)

In [ ]:
db = seispy.pandas.catalog.Catalog(
    "/home/malcolmw/google_drive/malcolm.white@usc.edu/data/events/malcolmw/SJFZ_catalog_2008-2016.h5",
    fmt="hdf5"
)

stations_dataframe = db["site"][
    ["sta", "lat", "lon", "elev"]
].drop_duplicates(
    "sta"
).merge(
    db["snetsta"][
        ["snet", "sta"]
    ].drop_duplicates(
        "sta"
    ),
    on="sta"
)
stations_dataframe["station_id"] = stations_dataframe["snet"] + "." + stations_dataframe["sta"]

arrivals_dataframe = db["arrival"][
    ["sta", "time", "arid"]
].merge(
    db["assoc"][
        ["arid", "orid", "phase"]
    ],
    on="arid"
).merge(
    stations_dataframe[["sta", "station_id"]],
    on="sta"
).merge(
    db["origin"][["orid", "evid"]],
    on="orid"
).sort_values(
    "evid"
).set_index(
    "evid"
)[
    ["station_id", "phase", "time", "arid"]
]

stations_dataframe["depth"] = -stations_dataframe["elev"]
stations_dataframe = stations_dataframe[
    ["station_id", "lat", "lon", "depth"]
].set_index(
    "station_id"
)
stations_dataframe = stations_dataframe[
    stations_dataframe.index.isin(arrivals_dataframe["station_id"].unique())
]

In [ ]:
stations_dict = dict(zip(stations_dataframe.index.values, geo2sph(stations_dataframe)))

In [ ]:
# Only events inside the focus region of White et al. 2019 with 128 or more arrivals
df = arrivals_dataframe.reset_index().groupby("evid").count()
arrivals_dataframe = arrivals_dataframe[
     (arrivals_dataframe.index.isin(df[df["station_id"] >= 128].index))
    &(arrivals_dataframe.index.isin(
        db["origin"][seispy.coords.as_geographic(db["origin"][["lat", "lon", "depth"]]).in_rectangle()]["evid"]
        )
     )
]

In [ ]:
def generate_arrivals(arrivals_dataframe):
    for event_id in arrivals_dataframe.index.unique()[:]:
        yield (event_id, arrivals_dataframe.loc[event_id].set_index(["station_id", "phase"]).to_dict()["time"])

In [ ]:
locator = pykonal.locate.EQLocator(stations_dict, tt_dir="/home/malcolmw/scratch/traveltimes")
locator.grid.min_coords = grid.min_coords
locator.grid.node_intervals = grid.node_intervals
locator.grid.npts = grid.npts
locator.pwave_velocity = pwave_velocity
locator.swave_velocity = swave_velocity
#     locator.compute_all_traveltime_lookup_tables("P")
#     locator.compute_all_traveltime_lookup_tables("S")
for event_id, arrivals in generate_arrivals(arrivals_dataframe):
    locator.clear_arrivals()
    locator.add_arrivals(arrivals)
    locator.load_traveltimes()
    %time loc0 = locator.grid_search()
    %time loc1 = locator.locate()
    break

In [ ]:
locator.counter

In [ ]:
# [locator.arrivals[key] for key in sorted(locator.arrivals)], 
[locator.traveltimes[key] for key in sorted(locator.arrivals)]

In [ ]:
locator.grid.max_coords, locator.grid.min_coords + locator.grid.node_intervals * (locator.grid.npts - 1)

In [ ]:
sph2geo(loc0[:3]), sph2geo(loc1[:3])

In [ ]:
locator.cost(loc0), locator.cost(loc1)

In [ ]:
%%cython

import numpy as np
import scipy.optimize
cimport numpy as np


ctypedef np.float64_t _REAL_t

EARTH_RADIUS = 6371.
DTYPE_REAL = np.float64

def geo2sph(arr):
    """
    Map Geographical coordinates to spherical coordinates.
    """
    geo = np.array(arr, dtype=DTYPE_REAL)
    sph = np.empty_like(geo)
    sph[..., 0] = EARTH_RADIUS - geo[..., 2]
    sph[..., 1] = np.pi / 2 - np.radians(geo[..., 0])
    sph[..., 2] = np.radians(geo[..., 1])
    return (sph)


def sph2geo(arr):
    """
    Map spherical coordinates to geographic coordinates.
    """
    sph = np.array(arr, dtype=DTYPE_REAL)
    geo = np.empty_like(sph)
    geo[..., 0] = np.degrees(np.pi / 2 - sph[..., 1])
    geo[..., 1] = np.degrees(sph[..., 2])
    geo[..., 2] = EARTH_RADIUS - sph[..., 0]
    return (geo)


cdef class EQLocator(object):
    cdef dict _arrivals
    cdef dict _tt_calculators
    cdef _REAL_t[:,:,:,:] _grid
    cdef tuple _bounds
    cdef dict _priors
    
    def __init__(self, arrivals, tt_calculators, grid):
        self.arrivals = {key: arrivals[key] for key in tt_calculators}
        self.tt_calculators = tt_calculators
        self.grid = grid
    
    @property
    def arrivals(self):
        return (self._arrivals)
    
    @arrivals.setter
    def arrivals(self, value):
        self._arrivals = value
    
    @property
    def grid(self):
        return (np.asarray(self._grid))
    
    @grid.setter
    def grid(self, value):
        self._grid = value
        
    @property
    def tt_calculators(self):
        return (self._tt_calculators)
    
    @tt_calculators.setter
    def tt_calculators(self, value):
        self._tt_calculators = value
        
    cpdef initial_guess(self):
        values = [self.arrivals[key]-self.tt_calculators[key].values for key in self.tt_calculators]
        values = np.ma.masked_invalid(np.stack(values))
        std = values.std(axis=0)
        arg_min = np.argmin(std)
        idx_min = np.unravel_index(arg_min, std.shape)
        geo = sph2geo(self.grid[idx_min])
        time = values.mean(axis=0)[idx_min]
        return (np.array([*geo, time]))

    cpdef cost(self, _REAL_t[:] hypocenter):
        cdef tuple key
        cdef _REAL_t csum = 0
        cdef _REAL_t lat, lon, depth, time
        lat = hypocenter[0]
        lon = hypocenter[1]
        depth = hypocenter[2]
        time = hypocenter[3]
        sph_coords = geo2sph((lat, lon, depth))
        if not np.all(sph_coords > np.asarray(self.grid).min(axis=(0,1,2))) or not np.all(sph_coords < np.asarray(self.grid).max(axis=(0,1,2))):
            return (np.inf)
        for key in self.arrivals:
            tt_calculator = self.tt_calculators[key].value
            csum += np.square(self.arrivals[key]-(time+tt_calculator(sph_coords)))
        return (np.sqrt(csum/len(self.arrivals)))
    
    cpdef locate(self):
        cdef _REAL_t[4] h0
        h0 = self.initial_guess()
        soln = scipy.optimize.differential_evolution(
            self.cost,
            ((h0[0]-0.1, h0[0]+0.1), (h0[1]-0.1, h0[1]+0.1), (0, 30), (h0[3]-5, h0[3]+5))
        )
        return (soln.x)

# Load a test data set

In [ ]:
db = seispy.pandas.catalog.Catalog(
    "/home/malcolmw/google_drive/malcolm.white@usc.edu/data/events/malcolmw/SJFZ_catalog_2008-2016.h5",
    fmt="hdf5"
)

gc_events_dataframe = pd.read_csv(
    "/home/malcolmw/scratch/events.grow", 
    delim_whitespace=True, 
    header=None,
    names=["year", "month", "day", "hour", "minute", "second", "lat", "lon", "depth", "null_0", "null_1", "null_2", "null_3", "event_id"]
)
gc_events_dataframe["time"] = seispy.pandas.time.to_epoch(seispy.pandas.time.ymd_to_dt(gc_events_dataframe))*1e-9
gc_events_dataframe["event_id"] -= int(2e9)
db["origin"] = db["origin"][["orid", "evid"]].merge(
    gc_events_dataframe[["lat", "lon", "depth", "time", "event_id"]],
    left_on="evid",
    right_on="event_id"
).drop(columns=["evid"])

stations_dataframe = db["site"][
    ["sta", "lat", "lon", "elev"]
].drop_duplicates(
    "sta"
).merge(
    db["snetsta"][
        ["snet", "sta"]
    ].drop_duplicates(
        "sta"
    ),
    on="sta"
)
stations_dataframe["station_id"] = stations_dataframe["snet"] + "." + stations_dataframe["sta"]

arrivals_dataframe = db["arrival"][
    ["sta", "time", "arid"]
].merge(
    db["assoc"][
        ["arid", "orid", "phase"]
    ],
    on="arid"
).merge(
    stations_dataframe[["sta", "station_id"]],
    on="sta"
).merge(
    db["origin"][["orid", "event_id"]],
    on="orid"
).sort_values(
    "event_id"
).set_index(
    "event_id"
)[
    ["station_id", "phase", "time", "arid"]
]

stations_dataframe["depth"] = -stations_dataframe["elev"]
stations_dataframe = stations_dataframe[
    ["station_id", "lat", "lon", "depth"]
].set_index(
    "station_id"
)
stations_dataframe = stations_dataframe[
    stations_dataframe.index.isin(arrivals_dataframe["station_id"].unique())
]

# Read and resample the velocity model

In [ ]:
with np.load("/home/malcolmw/google_drive/malcolm.white@usc.edu/data/velocity/White_et_al_2019a/White_et_al_2019a.regular.npz") as npz:
    vp = pykonal.field.ScalarField3D(coord_sys="spherical")
    vs = pykonal.field.ScalarField3D(coord_sys="spherical")
    vp.min_coords = vs.min_coords = npz["grid_parameters"][:3]
    vp.node_intervals = vs.node_intervals = npz["grid_parameters"][3:6]
    vp.npts = vs.npts = npz["grid_parameters"][6:] + [1, 0, 0]
    vp.values = np.append(npz["vp"], npz["vp"][[-1]], axis=0)
    vs.values = np.append(npz["vs"], npz["vs"][[-1]], axis=0)
    
# Resample the velocity model (18, 32, 32) --> (64, 128, 128)
rho_min, theta_min, phi_min = vp.min_coords
rho_max, theta_max, phi_max = vp.max_coords
nrho, ntheta, nphi = 64, 128, 128

drho = (rho_max - rho_min) / (nrho - 1)
rho = np.linspace(rho_min, rho_max, nrho)

dtheta = (theta_max - theta_min) / (ntheta - 1)
theta = np.linspace(theta_min, theta_max, ntheta)

dphi = (phi_max - phi_min) / (nphi - 1)
phi = np.linspace(phi_min, phi_max, nphi)

rtp = np.moveaxis(
    np.stack(np.meshgrid(rho, theta, phi, indexing="ij")),
    0, 
    -1
)
vp_new = vp.resample(rtp.reshape(-1, 3)).reshape(rtp.shape[:-1])
vs_new = vs.resample(rtp.reshape(-1, 3)).reshape(rtp.shape[:-1])

vp = pykonal.field.ScalarField3D(coord_sys="spherical")
vs = pykonal.field.ScalarField3D(coord_sys="spherical")
vp.min_coords = vs.min_coords = rho_min, theta_min, phi_min
vp.node_intervals = vs.node_intervals = drho, dtheta, dphi
vp.npts = vs.npts = nrho, ntheta, nphi
vp.values = vp_new
vs.values = vs_new

# Compute traveltime-lookup tables

In [ ]:
def target(args, vp=vp, vs=vs):
    station_id, coords = args[0], geo2sph(args[1].values)
    print(station_id)
    solver = compute_traveltime_lookup_table(coords, vp)
    solver.tt.savez(f"/home/malcolmw/scratch/traveltimes/{station_id}.P.npz")
    solver = compute_traveltime_lookup_table(coords, vs)
    solver.tt.savez(f"/home/malcolmw/scratch/traveltimes/{station_id}.S.npz")
    
with mp.Pool() as pool:
    pool.map(target, stations_dataframe.iterrows())

# Locate events

In [ ]:
def generate_arrivals(arrivals_dataframe):
    for event_id in arrivals_dataframe.index.unique()[:]:
        yield (event_id, arrivals_dataframe.loc[event_id].set_index(["station_id", "phase"]).to_dict()["time"])
    
def target(args, nodes=vp.nodes):
    event_id, arrivals = args
    print(f"Locating event #{event_id}")
    tt_calculators = {
        key: pykonal.field.load(f"/home/malcolmw/scratch/traveltimes/{key[0]}.{key[1]}.npz") for key in arrivals
    }
    locator = EQLocator(
        arrivals=arrivals,
        tt_calculators=tt_calculators,
        grid=nodes
    )
    return (event_id, locator.locate())

In [ ]:
with mp.Pool(4) as pool:
    locations = pool.map(target,  generate_arrivals(arrivals_dataframe))

In [ ]:
relocated_events_dataframe = pd.DataFrame([[loc[0], *loc[1]] for loc in locations], columns=["event_id", "lat", "lon", "depth", "time"]).set_index("event_id")

In [ ]:
original_events_dataframe = db["origin"][db["origin"]["event_id"].isin(arrivals_dataframe.index.unique())].set_index("event_id")

In [ ]:
plt.close("all")
fig = plt.figure()
ax00 = fig.add_subplot(2, 2, 1)
ax01 = fig.add_subplot(2, 2, 2)
ax10 = fig.add_subplot(2, 2, 3)
ax11 = fig.add_subplot(2, 2, 4)
for ax, key in ((ax00, "lat"), (ax01, "lon"), (ax10, "depth"), (ax11, "time")):
    ax.hist(original_events_dataframe[key] - relocated_events_dataframe[key], bins=32)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(1, 3, 1, aspect=1)
ax2 = fig.add_subplot(1, 3, 2, aspect=1)
ax3 = fig.add_subplot(1, 3, 3, aspect=1)
ax1.scatter(
    original_events_dataframe["lon"], 
    original_events_dataframe["lat"],
    c=original_events_dataframe["depth"],
    vmin=0,
    vmax=25
)
ax2.scatter(
    relocated_events_dataframe["lon"],
    relocated_events_dataframe["lat"],
    c=original_events_dataframe["depth"],
    vmin=0,
    vmax=25
)
ax3.quiver(
    original_events_dataframe["lon"], 
    original_events_dataframe["lat"],
    relocated_events_dataframe["lon"] - original_events_dataframe["lon"],
    relocated_events_dataframe["lat"] - original_events_dataframe["lat"],
)
for ax in (ax1, ax2, ax3):
    ax.set_xlim(-117, -116)
    ax.set_ylim(33, 34)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect=1)
ax.quiver(
    original_events_dataframe["lon"], 
    original_events_dataframe["lat"],
    relocated_events_dataframe["lon"] - original_events_dataframe["lon"],
    relocated_events_dataframe["lat"] - original_events_dataframe["lat"],
)

In [ ]:
for event_id, origin in relocated_events_dataframe.iterrows():
    arrivals = arrivals_dataframe.loc[event_id].set_index(["station_id", "phase"]).to_dict()["time"]
    tt_calculators = {
        key: pykonal.field.load(f"/home/malcolmw/scratch/traveltimes/{key[0]}.{key[1]}.npz") for key in arrivals
    }
    locator = EQLocator(
        arrivals=arrivals,
        tt_calculators=tt_calculators,
        grid=vp.nodes
    )
    print(
        event_id, 
        locator.cost(origin[["lat", "lon", "depth", "time"]].values), 
        locator.cost(original_events_dataframe.loc[event_id, ["lat", "lon", "depth", "time"]].values)
    )